In [1]:
import pandas
import mido
import os
import re
import shutil

In [2]:
# Use this to strip titles and instrument data of non-alphabetic characters
pattern = "[^a-zA-Z]+"

In [3]:
import os
os.getcwd()
raw_data_folder = os.path.join(os.pardir, "raw_data", "Unzipped")
#raw_data_folder = os.path.join(os.pardir, "test_data")
output_folder = os.path.join(os.pardir, "data")
raw_data_folder

'../raw_data/Unzipped'

In [4]:
already_saved = os.listdir(output_folder)

In [5]:
def identify_drum_and_guitar(midi):
    guitars = {"guitar", "guita", "gtr", "egtr", "eguitar"}
    drums = {"drums", "drum", "drumsperc", "perc", "percussion"}

    pattern = "[^a-zA-Z ]+"
    has_drum = False
    has_guitar = False

    for track in midi:
        # Check if instruments in track name
        if track.type == "track_name":
            words = {re.sub(pattern, '', word.lower()) for word in track.name.split()}
            if words.intersection(guitars):
                has_guitar = True
            if words.intersection(drums):
                has_drum = True
    if has_drum and has_guitar:
        return True

    # Check if instruments in track text
    for track in midi:
        if track.type == "text":
            words = {re.sub(pattern, '', word.lower())  for word in track.text.split()}
            if words.intersection(guitars):
                has_guitar = True
            if words.intersection(drums):
                has_drum = True
    if has_drum and has_guitar:
        return True

In [6]:
%time

saved_songs = []
songs = []
errors = 0

for idx, child in enumerate(os.listdir(raw_data_folder)):
    if (idx == 0) or (idx % 250 == 0):
        print(f"Processing file number: {idx}")

    # If the path is a midi file, load the file
    child_path = os.path.join(raw_data_folder, child)
    try:
        if child_path.endswith(".mid") and child not in already_saved:
            song_path = child_path
            midi_file = mido.MidiFile(song_path, clip=True)
            dg_song = identify_drum_and_guitar(midi_file)
            if dg_song:
                title = re.sub(pattern, "", child[:-4]).lower()
                if title not in saved_songs:
                    saved_songs.append(title)
                    #midi_file.save(f'{output_folder}/{child}')
                    shutil.copyfile(song_path, f'{output_folder}/{child}')


        # If the path is a folder, load the midi files inside
        elif os.path.isdir(child_path):
            for child in os.listdir(child_path):
                song_path = os.path.join(child_path, child)
                if song_path.endswith(".mid") and child not in already_saved:
                    midi_file = mido.MidiFile(song_path, clip=True)
                    dg_song = identify_drum_and_guitar(midi_file)
                    if dg_song:
                        title = re.sub(pattern, "", child[:-4]).lower()
                        if title not in saved_songs:
                            saved_songs.append(title)
                            #midi_file.save(f'{output_folder}/{child}')
                            shutil.copyfile(song_path, f'{output_folder}/{child}')

    except Exception as ke:
        errors +=1

print(f"Processed {idx+1} files in {raw_data_folder}.")
print(f"Encountered {errors} errors.")
print("Number of songs already in data folder from a previous run:", len(already_saved))
print("Number of new songs saved in data folder in this run", len(saved_songs))
print("Total number of non-duplicate songs with drums and guitar saved:", len(saved_songs) + len(already_saved))

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 21.5 µs
Processing file number: 0
Processing file number: 250
Processing file number: 500
Processing file number: 750
Processing file number: 1000
Processing file number: 1250
Processing file number: 1500
Processing file number: 1750
Processing file number: 2000
Processing file number: 2250
Processing file number: 2500
Processing file number: 2750
Processing file number: 3000
Processing file number: 3250
Processing file number: 3500
Processing file number: 3750
Processing file number: 4000
Processing file number: 4250
Processing file number: 4500
Processing file number: 4750
Processing file number: 5000
Processing file number: 5250
Processing file number: 5500
Processing file number: 5750
Processing file number: 6000
Processing file number: 6250
Processing file number: 6500
Found 1108 .mid files in ../raw_data/Unzipped.
Encountered 77 errors.


In [11]:
print(f"Processed {idx+1} files in {raw_data_folder}.")
print(f"Encountered {errors} errors.")
print("Number of songs already in data folder from a previous run:", len(already_saved))
print("Number of new songs saved in data folder in this run", len(saved_songs))
print("Total number of non-duplicate songs with drums and guitar saved:", len(saved_songs) + len(already_saved))

Processed 6695 files in ../raw_data/Unzipped.
Encountered 77 errors.
Number of songs already in data folder from a previous run: 405
Number of new songs saved in data folder in this run 1108
Total number of non-duplicate songs with drums and guitar saved: 1513
